In [ ]:
pip install feedparser beautifulsoup4 pandas scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6047 sha256=716c0a63146f92b9651add80514818a0c6424b6db5e20c4332b53f5b3de554d7
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k


In [ ]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
#import RSS feed parsing library Feedparser
import feedparser
import pandas as pd

def feedparser_basic_process(rss_url, description):

  # Choose an RSS feed URL from a news website
  # find the RSS URL for global mails

  #process with feedparser
  feed = feedparser.parse(rss_url)
  posts = []

  if not description:
    for entry in feed.entries:
    #add the entires of the feed to the list
      posts.append((entry.title, entry.link, entry.published, entry.id))
    df = pd.DataFrame(posts, columns=['title', 'link', 'published', 'id'])
    return df

  for entry in feed.entries:
    #add the entires of the feed to the list
    posts.append((entry.title, entry.link, entry.description, entry.published, entry.id))

  # Convert to DataFrame
  df = pd.DataFrame(posts, columns=['title', 'link', 'description', 'published', 'id'])

  return df

In [ ]:
from bs4 import BeautifulSoup
import requests

def scrape_article(url):
    content = ""
    try:
        # Use requests to fetch the content of the URL
        page = requests.get(url)
        # Use Beautiful Soup to parse the content
        soup = BeautifulSoup(page.content, 'html.parser')

        for paragraph in soup.find_all('p'):
            content += paragraph.get_text()
    except Exception as e:
        print(f"An error occurred: {e}")
    return content


In [ ]:
def scrape_article_only_news_content(url):
    content = ""
    try:
        # Use requests to fetch the content of the URL
        page = requests.get(url)
        # Use Beautiful Soup to parse the content

        soup = BeautifulSoup(page.content, 'html.parser')
        text_blocks = soup.find_all('div', attrs={'data-component':'text-block'})
        for block in text_blocks:
            paragraphs= block.find_all('p')
            for paragraph in paragraphs:
              content+= paragraph.get_text()
    except Exception as e:
        print(f"An error occurred: {e}")
    return content

In [ ]:
def combine_data(rss_url,description):
  df = feedparser_basic_process(rss_url,description)
  # Append the full text to our DataFrame, the link is the website to be scrapped
  df['full_text'] = df['link'].apply(scrape_article)
  return df

In [ ]:
df_gbnews = feedparser_basic_process('https://www.theglobeandmail.com/arc/outboundfeeds/rss/category/canada/',True)
# Append the full text to our DataFrame, the link is the website to be scrapped
df_gbnews['full_text'] = df_gbnews['link'].apply(scrape_article_only_news_content)
df_gbnews.to_csv('gbnews.csv', index=False)
df_gbnews

<bound method NDFrame.head of                                                 title  \
0   Quebec public sector unions say they will move...   
1   New evidence confirms COVID-19 vaccines are ov...   
2   CRTC urged to mandate better supports for Deaf...   
3   Gen Z is a climate-anxious, pessimistic force ...   
4   Sikh independence vote takes place in B.C. ami...   
..                                                ...   
95  Evening Update: Netanyahu says Israel preparin...   
96  Beyond tightening, where is the end point of t...   
97  Ontario mayors say rules for provincial housin...   
98  Tiff Macklem outlines reasons for holding poli...   
99  What to make of Alberta’s unrealistic CPP with...   

                                                 link  \
0   https://www.theglobeandmail.com/canada/article...   
1   https://www.theglobeandmail.com/canada/article...   
2   https://www.theglobeandmail.com/business/artic...   
3   https://www.theglobeandmail.com/canada/article...   


In [ ]:
#select five rss url from
#Globe and Mail:'https://www.theglobeandmail.com/arc/outboundfeeds/rss/category/canada/'
#New York Times:'https://rss.nytimes.com/services/xml/rss/nyt/World.xml'
#CBC News: 'https://www.cbc.ca/webfeed/rss/rss-canada'
#BBC News: 'https://feeds.bbci.co.uk/news/world/us_and_canada/rss.xml'

df_globe = combine_data('https://www.theglobeandmail.com/arc/outboundfeeds/rss/category/canada/', True)
df_globe.to_csv('news_articles_globeandmail.csv', index=False)

df_NY = combine_data('https://rss.nytimes.com/services/xml/rss/nyt/World.xml', True)
df_NY.to_csv('news_articles_NY.csv', index=False)

df_CBC = combine_data('https://www.cbc.ca/webfeed/rss/rss-canada', True)
df_CBC.to_csv('news_articles_CBC.csv', index=False)

df_BBC = combine_data('https://feeds.bbci.co.uk/news/world/us_and_canada/rss.xml', True)
df_BBC.to_csv('news_articles_BBC.csv', index=False)

In [ ]:
df = pd.concat([df_globe, df_NY, df_CBC, df_BBC])

In [ ]:
df.to_csv('All_news.csv', index=False)
df.head(5)

title  \
0  Quebec public sector unions say they will move...   
1  New evidence confirms COVID-19 vaccines are ov...   
2  CRTC urged to mandate better supports for Deaf...   
3  Gen Z is a climate-anxious, pessimistic force ...   
4  Sikh independence vote takes place in B.C. ami...   

                                                link  \
0  https://www.theglobeandmail.com/canada/article...   
1  https://www.theglobeandmail.com/canada/article...   
2  https://www.theglobeandmail.com/business/artic...   
3  https://www.theglobeandmail.com/canada/article...   
4  https://www.theglobeandmail.com/canada/british...   

                                         description  \
0  Quebec public sector unions federations repres...   
1  More than 38 million COVID-19 vaccine doses ha...   
2  Rogers outage in July, 2022, was far more dire...   
3  Young Canadians are fearful about the future. ...   
4  Organizers say the referendum on Khalistan – a...   

                         published  \
0  Sun, 29 Oct 2023 11:47:43 +0000   
1  Sat, 28 Oct 2023 14:00:00 +0000   
2  Sun, 29 Oct 2023 14:32:43 +0000   
3  Sat, 28 Oct 2023 12:45:00 +0000   
4  Sun, 29 Oct 2023 11:45:19 +0000   

                                                  id  \
0  https://www.theglobeandmail.com/canada/article...   
1  https://www.theglobeandmail.com/canada/article...   
2  https://www.theglobeandmail.com/business/artic...   
3  https://www.theglobeandmail.com/canada/article...   
4  https://www.theglobeandmail.com/canada/british...   

                                           full_text  
0  People take part in a public sector union demo...  
1  As updated COVID-19 vaccines are rolled out ac...  
2  As Canada’s telecommunications regulator seeks...  
3  Young Canadians are fearful about the future. ...  
4  A photograph of late temple president Hardeep ...

In [ ]:
df_new = df.iloc[7:11].reset_index(drop=True)
df_new

title  \
0  Police watchdog asks Crown to consider charges...   
1  Shooter in Sault Ste. Marie killings identifie...   
2  How a scientist got a Toronto suburb to protec...   
3  Location, location, location: three former Hel...   

                                                link  \
0  https://www.theglobeandmail.com/canada/british...   
1  https://www.theglobeandmail.com/canada/article...   
2  https://www.theglobeandmail.com/canada/article...   
3  https://www.theglobeandmail.com/canada/article...   

                                         description  \
0  A statement from B.C.’s Independent Investigat...   
1  Court-imposed conditional sentence against Bob...   
2  David Lawrie has spent years studying and defe...   
3  They are being added to a $155-million bounty ...   

                         published  \
0  Sat, 28 Oct 2023 18:42:19 +0000   
1  Fri, 27 Oct 2023 00:22:31 +0000   
2  Sat, 28 Oct 2023 13:00:00 +0000   
3  Sat, 28 Oct 2023 12:21:26 +0000   

                                                  id  \
0  https://www.theglobeandmail.com/canada/british...   
1  https://www.theglobeandmail.com/canada/article...   
2  https://www.theglobeandmail.com/canada/article...   
3  https://www.theglobeandmail.com/canada/article...   

                                           full_text  
0  The RCMP logo is seen outside Royal Canadian M...  
1  Brian T. Sweeney, the father of murder victim ...  
2  A juvenile Jefferson salamander crosses Stouff...  
3  A former Hells Angels clubhouse is pictured in...

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

def preprocess_text(text):
    # Lowercasing
    text = text.lower()
    # Tokenization (split the text into words)
    words = word_tokenize(text)
    # Removing punctuation and special characters
    words = [word for word in words if word.isalnum()]
    # Removing stopwords and words containing only digits
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words and not word.isdigit()]
    return ' '.join(words)

full_text = df_new['full_text'].apply(preprocess_text)
full_text[2]

'rcmp logo seen outside royal canadian mounted police e division headquarters surrey april canadian pressinvestigators british columbia independent police watchdog probing shooting death man two years ago asking province prosecution service consider charges three rcmp statement independent investigations office says submitted report prosecution service consideration charges related july incident campbell river vancouver statement says ronald macdonald iio chief civilian director reviewed evidence determined reasonable grounds believe three officers may committed offences relation various uses says interaction took place man police window local business police fired shots driver vehicle reports identified victim jared lowndes campbell statement says order approve charges prosecution service must satisfied substantial likelihood conviction based evidence gathered iio prosecution public errorbuild personal news feedbuild personal news copyright globe mail rights saunders president ceo'

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
ngram_vectorizer = CountVectorizer(ngram_range=(1, 1))
X_ngrams = ngram_vectorizer.fit_transform(full_text)
# Create a DataFrame for the TF-IDF matrix
ngram_df = pd.DataFrame(X_ngrams.toarray(), columns=ngram_vectorizer.get_feature_names_out())

# Display the 1-ngram
ngram_df

80s  able  abuse  accidentally  accommodation  according  acquired  across  \
0    0     0      0             1              0          0         0       0   
1    1     1      0             0              1          1         0       0   
2    0     0      0             0              0          0         0       0   
3    0     0      1             0              0          0         1       1   

   additional  address  ...  worked  working  worst  would  years  yet  yoho  \
0           0        0  ...       0        0      0      0      0    0     1   
1           0        0  ...       0        1      1      0      0    0     0   
2           0        0  ...       0        0      0      0      1    0     0   
3           1        1  ...       1        0      0      1      4    2     0   

   york  younger  yunis  
0     0        1      0  
1     1        0      1  
2     0        0      0  
3     0        0      0  

[4 rows x 646 columns]

In [ ]:
import numpy as np
n_docs = len(full_text)  # Number of documents in the corpus
words_set = set()
# TF caculate
for doc in full_text:
    words = doc.split(' ')
    words_set = words_set.union(set(words))

# Convert the set of unique words to a sorted list
words_list = sorted(list(words_set))

# Initialize the DataFrame with columns ordered according to words_list
df_tf = pd.DataFrame(np.zeros((n_docs, len(words_list))), columns=words_list)

# Compute Term Frequency (TF)
for i in range(n_docs):
    words = full_text[i].split(' ')  # Words in the document
    for w in words:
        df_tf.at[i, w] = df_tf.at[i, w] + (1 / len(words))

# Display the DataFrame with columns in the desired order
df_tf

80s      able     abuse  accidentally  accommodation  according  \
0  0.000000  0.000000  0.000000      0.007246       0.000000   0.000000   
1  0.002747  0.002747  0.000000      0.000000       0.002747   0.002747   
2  0.000000  0.000000  0.000000      0.000000       0.000000   0.000000   
3  0.000000  0.000000  0.002304      0.000000       0.000000   0.000000   

   acquired    across  additional   address  ...    worked   working  \
0  0.000000  0.000000    0.000000  0.000000  ...  0.000000  0.000000   
1  0.000000  0.000000    0.000000  0.000000  ...  0.000000  0.002747   
2  0.000000  0.000000    0.000000  0.000000  ...  0.000000  0.000000   
3  0.002304  0.002304    0.002304  0.002304  ...  0.002304  0.000000   

      worst     would     years       yet      yoho      york   younger  \
0  0.000000  0.000000  0.000000  0.000000  0.007246  0.000000  0.007246   
1  0.002747  0.000000  0.000000  0.000000  0.000000  0.002747  0.000000   
2  0.000000  0.000000  0.008000  0.000000  0.000000  0.000000  0.000000   
3  0.000000  0.002304  0.009217  0.004608  0.000000  0.000000  0.000000   

      yunis  
0  0.000000  
1  0.002747  
2  0.000000  
3  0.000000  

[4 rows x 647 columns]

In [ ]:
print("IDF of: ")

idf = {}

for w in words_set:
    k = 0    # number of documents in the corpus that contain this word

    for i in range(n_docs):
        if w in full_text[i].split():
            k += 1

    idf[w] =  np.log10(n_docs / k)

    print(f'{w:>15}: {idf[w]:>10}' )

IDF of: 
           rcmp: 0.6020599913279624
        partner: 0.6020599913279624
       children: 0.6020599913279624
      resisting: 0.6020599913279624
    governments: 0.6020599913279624
           food: 0.3010299956639812
         listed: 0.6020599913279624
     expressing: 0.6020599913279624
          marie: 0.6020599913279624
           pond: 0.6020599913279624
       romantic: 0.6020599913279624
          phone: 0.6020599913279624
        monarch: 0.6020599913279624
      president:        0.0
  consideration: 0.6020599913279624
           noon: 0.6020599913279624
          heard: 0.6020599913279624
         sunday: 0.6020599913279624
        disease: 0.6020599913279624
        working: 0.6020599913279624
           case: 0.3010299956639812
          march: 0.6020599913279624
          cases: 0.6020599913279624
       recently: 0.3010299956639812
           risk: 0.6020599913279624
   conversation: 0.6020599913279624
          weeks: 0.6020599913279624
           lone: 0.60205999

In [ ]:
## TF-IDF by calcualte
df_tf_idf = df_tf.copy()

for w in words_set:
    for i in range(n_docs):
        df_tf_idf[w][i] = df_tf[w][i] * idf[w]

df_tf_idf


80s      able     abuse  accidentally  accommodation  according  \
0  0.000000  0.000000  0.000000      0.004363       0.000000   0.000000   
1  0.001654  0.001654  0.000000      0.000000       0.001654   0.001654   
2  0.000000  0.000000  0.000000      0.000000       0.000000   0.000000   
3  0.000000  0.000000  0.001387      0.000000       0.000000   0.000000   

   acquired    across  additional   address  ...    worked   working  \
0  0.000000  0.000000    0.000000  0.000000  ...  0.000000  0.000000   
1  0.000000  0.000000    0.000000  0.000000  ...  0.000000  0.001654   
2  0.000000  0.000000    0.000000  0.000000  ...  0.000000  0.000000   
3  0.001387  0.001387    0.001387  0.001387  ...  0.001387  0.000000   

      worst     would     years       yet      yoho      york   younger  \
0  0.000000  0.000000  0.000000  0.000000  0.004363  0.000000  0.004363   
1  0.001654  0.000000  0.000000  0.000000  0.000000  0.001654  0.000000   
2  0.000000  0.000000  0.002408  0.000000  0.000000  0.000000  0.000000   
3  0.000000  0.001387  0.002774  0.002774  0.000000  0.000000  0.000000   

      yunis  
0  0.000000  
1  0.001654  
2  0.000000  
3  0.000000  

[4 rows x 647 columns]

In [ ]:
# TF-IDF representation
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(full_text)

# Create a DataFrame for the TF-IDF matrix
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Display the TF-IDF DataFrame
tfidf_df

80s      able     abuse  accidentally  accommodation  according  \
0  0.000000  0.000000  0.000000      0.071563       0.000000   0.000000   
1  0.035624  0.035624  0.000000      0.000000       0.035624   0.035624   
2  0.000000  0.000000  0.000000      0.000000       0.000000   0.000000   
3  0.000000  0.000000  0.033484      0.000000       0.000000   0.000000   

   acquired    across  additional   address  ...    worked   working  \
0  0.000000  0.000000    0.000000  0.000000  ...  0.000000  0.000000   
1  0.000000  0.000000    0.000000  0.000000  ...  0.000000  0.035624   
2  0.000000  0.000000    0.000000  0.000000  ...  0.000000  0.000000   
3  0.033484  0.033484    0.033484  0.033484  ...  0.033484  0.000000   

      worst     would     years       yet      yoho      york   younger  \
0  0.000000  0.000000  0.000000  0.000000  0.071563  0.000000  0.071563   
1  0.035624  0.000000  0.000000  0.000000  0.000000  0.035624  0.000000   
2  0.000000  0.000000  0.064499  0.000000  0.000000  0.000000  0.000000   
3  0.000000  0.033484  0.105595  0.066967  0.000000  0.000000  0.000000   

      yunis  
0  0.000000  
1  0.035624  
2  0.000000  
3  0.000000  

[4 rows x 646 columns]

In [39]:
import nltk
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
print(stop_words)


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '